# Update RBC model

The purpose of this notebook is to update the iAB-RBC-283 reconstruction for the following:
1. Base model for RBC reconstruction expansion
2. Cleaned up and uplifted model to use for comparison to new reconstruction

Bordbar, A., Jamshidi, N. & Palsson, B.O. iAB-RBC-283: A proteomically derived knowledge-base of erythrocyte metabolism that can be used to simulate its physiological and patho-physiological states. BMC Syst Biol 5, 110 (2011). https://doi.org/10.1186/1752-0509-5-110


## Setup
### Import packages

In [1]:
from operator import attrgetter
from collections import Counter

import cobra
from cobra import Configuration
from cobra.io.json import load_json_model, save_json_model
from cobra.io.sbml import read_sbml_model, write_sbml_model
from cobra.io.yaml import save_yaml_model
from cobra.io.mat import save_matlab_model

import irbc_gem
from irbc_gem import MAIN_PROJECT_DIR
    
irbc_gem.__version__

'0.0.1'

### Define configuration
#### COBRA Configuration

In [2]:
COBRA_CONFIGURATION = Configuration()
COBRA_CONFIGURATION.solver = "gurobi"
COBRA_CONFIGURATION

Attribute,Description,Value
solver,Mathematical optimization solver,gurobi
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,15
cache_directory,Path for the model cache,/Users/zhaiman/Library/Caches/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


## Load iAB-RBC-283

In [3]:
# JSON contains subsystems, SBML contains annotations
iAB_RBC_283_json = load_json_model(f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/iAB_RBC_283_bigg_10_31_2019.json")
iAB_RBC_283_sbml = read_sbml_model(f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/iAB_RBC_283_bigg_10_31_2019.xml")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-26


In [4]:
# Load pruned models
Recon3D = read_sbml_model(f"{MAIN_PROJECT_DIR}/models/cobra/Recon3D_pruned.xml")
HumanGEM = read_sbml_model(f"{MAIN_PROJECT_DIR}/models/cobra/HumanGEM_pruned.xml")

No objective coefficients in model. Unclear what should be optimized


## Update model with subsystems

In [5]:
model = iAB_RBC_283_sbml.copy()
model

Read LP format model from file /var/folders/5t/hk8m3g6d1jn25x5rssjgsrmm0000gn/T/tmpxnb59op7.lp
Reading time = 0.01 seconds
: 342 rows, 938 columns, 3344 nonzeros


Name,iAB_RBC_283
Memory address,7fa5a1b8b9d0
Number of metabolites,342
Number of reactions,469
Number of genes,346
Number of groups,0
Objective expression,1.0*NaKt - 1.0*NaKt_reverse_db47e
Compartments,"cytosol, extracellular space"


In [6]:
subsystems_set = set()
for reaction in model.reactions:
    reaction.subsystem = iAB_RBC_283_json.reactions.get_by_id(reaction.id).subsystem
    subsystems_set.add(reaction.subsystem)
subsystems_set

{'&apos;',
 'Aminosugar Metabolism',
 'Arginine and Proline Metabolism',
 'Ascorbate and Aldarate Metabolism',
 'Carnitine shuttle',
 'Citric Acid Cycle',
 'Diacylglycerol Synthesis',
 'Eicosanoid Metabolism',
 'Extracellular exchange',
 'Fatty acid activation',
 'Fructose and Mannose Metabolism',
 'Galactose metabolism',
 'Glutamate metabolism',
 'Glutathione Metabolism',
 'Glycerophospholipid Metabolism',
 'Glycolysis/Gluconeogenesis',
 'Heme Biosynthesis',
 'Heme Degradation',
 'Inositol Phosphate Metabolism',
 'Intracellular source/sink',
 'Methionine Metabolism',
 'Methionine Salvage',
 'Miscellaneous',
 'NAD Metabolism',
 'Nucleotides',
 'Oxidative Phosphorylation',
 'Pentose Phosphate Pathway',
 'Pentose and Glucuronate Interconversions',
 'Phenylalanine metabolism',
 'Purine Catabolism',
 'Pyrimidine Biosynthesis',
 'Pyrimidine Catabolism',
 'Pyruvate Metabolism',
 'ROS Detoxification',
 'Riboflavin Metabolism',
 'Salvage Pathway',
 'Starch and Sucrose Metabolism',
 'Thiamine M

In [7]:
HumanGEM.groups.list_attr("name")

['Acyl-CoA hydrolysis',
 'Acylglycerides metabolism',
 'Alanine, aspartate and glutamate metabolism',
 'Alkaloids biosynthesis',
 'Amino sugar and nucleotide sugar metabolism',
 'Aminoacyl-tRNA biosynthesis',
 'Androgen metabolism',
 'Arachidonic acid metabolism',
 'Arginine and proline metabolism',
 'Artificial reactions',
 'Ascorbate and aldarate metabolism',
 'Beta oxidation of branched-chain fatty acids (mitochondrial)',
 'Beta oxidation of di-unsaturated fatty acids (n-6) (mitochondrial)',
 'Beta oxidation of di-unsaturated fatty acids (n-6) (peroxisomal)',
 'Beta oxidation of even-chain fatty acids (mitochondrial)',
 'Beta oxidation of even-chain fatty acids (peroxisomal)',
 'Beta oxidation of odd-chain fatty acids (mitochondrial)',
 'Beta oxidation of odd-chain fatty acids (peroxisomal)',
 'Beta oxidation of phytanic acid (peroxisomal)',
 'Beta oxidation of poly-unsaturated fatty acids (mitochondrial)',
 'Beta oxidation of unsaturated fatty acids (n-7) (mitochondrial)',
 'Beta o

## Update subsystems
Map them to new names seen in Human-GEM v1.15.0.

In [8]:
mapping = {
    '&apos;': 'Miscellaneous',
    'Aminosugar Metabolism': 'Amino sugar and nucleotide sugar metabolism',
    'Arginine and Proline Metabolism': 'Arginine and proline metabolism',
    'Ascorbate and Aldarate Metabolism': 'Ascorbate and aldarate metabolism',
    'Carnitine shuttle': 'Carnitine shuttle (cytosolic)',
    'Citric Acid Cycle': 'Tricarboxylic acid cycle and glyoxylate/dicarboxylate metabolism',
#     'Diacylglycerol Synthesis': Requires manual mapping,
    'Eicosanoid Metabolism': 'Eicosanoid metabolism', 
    'Extracellular exchange': 'Exchange/demand reactions',
    'Fatty acid activation': 'Fatty acid activation (cytosolic)',
    'Fructose and Mannose Metabolism': 'Fructose and mannose metabolism',
    'Glutamate metabolism': 'Alanine, aspartate and glutamate metabolism',
    'Glutathione Metabolism': 'Glutathione metabolism',
    'Glycerophospholipid Metabolism': 'Glycerophospholipid metabolism',
    'Glycolysis/Gluconeogenesis': 'Glycolysis / Gluconeogenesis',
    'Heme Biosynthesis': 'Heme synthesis',
    'Heme Degradation': 'Heme degradation',
    'Inositol Phosphate Metabolism': 'Inositol phosphate metabolism',
    'Intracellular source/sink': 'Exchange/demand reactions',
    'Methionine Metabolism': 'Cysteine and methionine metabolism',
    'Methionine Salvage': 'Cysteine and methionine metabolism',
    'NAD Metabolism': 'Nicotinate and nicotinamide metabolism',
    'Nucleotides': 'Nucleotide metabolism',
    'Oxidative phosphorylation': 'Oxidative phosphorylation',
    'Pentose Phosphate Pathway': 'Pentose phosphate pathway',
    'Pentose and Glucuronate Interconversions': 'Pentose and glucuronate interconversions',
#     'Phenylalanine metabolism': ,
    'Purine Catabolism': 'Purine metabolism',
    'Pyrimidine Biosynthesis': 'Pyrimidine metabolism',
    'Pyrimidine Catabolism': 'Pyrimidine metabolism',
    'Pyruvate Metabolism': 'Pyruvate metabolism',
    'ROS Detoxification': 'ROS detoxification',
    'Riboflavin Metabolism': 'Riboflavin metabolism',
    'Salvage Pathway': 'Purine metabolism',
#     'Starch and Sucrose Metabolism': ,
    'Thiamine Metabolism': 'Thiamine metabolism',
#     'Transport, Extracellular': ,
#     'Tyrosine metabolism': ,
    'Urea cycle/amino group metabolism': 'Urea cycle',
    'Vitamin B6 Metabolism': 'Vitamin B6 metabolism',
}
for reaction in model.reactions:
    reaction.subsystem = mapping.get(reaction.subsystem, reaction.subsystem)

### Genes
1. Update gene IDs to match RECON3D with inclusion of NCBI Gene ID .
2. Manually update genes without NCBI numbers and genes lumped together.
3. Update gene names to match annotation "refseq_name".

#### Generate ID map

In [9]:
to_check = {}
gene_mapping = {}

for gene in model.genes:
    ncbi = gene.annotation.get("ncbigene")
    
    if ncbi is None:
        to_check[gene] = ncbi
        continue
    if isinstance(ncbi, list):
        if len(ncbi) > 1:
            raise ValueError
            
        ncbi = ncbi[0]
    gsplit = gene.id.split("_")
    
    if len(gsplit) <= 1:
        to_check[gene] = ncbi
        continue
    new_gene = "_".join((ncbi, gsplit[-1]))
    gene_mapping[gene.id] = new_gene

##### To double check manually

In [10]:
# Assume canonical isoform only
gene_mapping["9429"] = "9429_AT1"
gene_mapping["55256"] = "55256_AT1"
gene_mapping["58478"] = "58478_AT1"
gene_mapping["55500"] = "55500_AT1"
gene_mapping["29124"] = "29124_AT1"
gene_mapping["10846"] = "10846_AT1"
gene_mapping["55276"] = "55276_AT1"
gene_mapping["51084"] = "51084_AT1"

In [11]:
# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["ThmtP_AT1"] = "79178_AT1"
# Mapped in Recon3D but not iAB-RBC-283
# NCBI says located in the nucleus, reactome says otherwise
gene_mapping["Cgi_14_AT1"] = "51005_AT1"
# Pseudogene, does not seem to have annotations mapped 
gene_mapping["Gkp3_AT1"] = "2713_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Rhag_1_e"] = "6005_AT1"
gene_mapping["Rhbg_1_e"] = "57127_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Abcc4_1_e"] = "10257_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc2a1_1_e"] = "6513_AT1"
gene_mapping["Slc2a2_1_e"] = "6514_AT1"
gene_mapping["Slc2a3_1_e"] = "6515_AT1"
gene_mapping["Slc2a4_1_e"] = "6517_AT1"
gene_mapping["Slc2a5_1_e"] = "6518_AT1"
gene_mapping["Slc2a7_1_e"] = "155184_AT1"
gene_mapping["Slc2a8_1_e"] = "29988_AT1"
gene_mapping["Slc2a11_1_e"] = "66035_AT1"


# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc4a1_1_e"] = "6521_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc5a1_1_e"] = "6523_AT1"
gene_mapping["Slc5a3_1_e"] = "6526_AT1"
gene_mapping["Slc5a5_1_e"] = "6528_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc12a7_1_e"] = "10723_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc14a1_1_e"] = "6563_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Slc29a1_1_e"] = "2030_AT1"
gene_mapping["Slc29a2_1_e"] = "3177_AT1"

# Mapped in Recon3D but not iAB-RBC-283
gene_mapping["Flj22761_AT1"] = "80201_AT1"

##### Map new IDs onto genes

In [12]:
cobra.manipulation.modify.rename_genes(model, gene_mapping)

##### Seperate "Lumped" genes into individual genes
Note: These are reactions containing GPRs that have both "AND" and "OR" in them. The "AND" genes are lumped together.

In [13]:
# Reaction GUACYC: Recon3D top, iAB-RBC-283 bottom
# 4881_AT1 or 4882_AT1 or 4882_AT2 or   2984_AT1 or   3000_AT1 or   2986_AT1 or (2977_AT1 and 2974_AT1) or (2977_AT1 and 2983_AT1) or (2982_AT1 and 2974_AT1) or (2982_AT1 and 2983_AT1)
# Npr1_AT1 or Npr2_AT1 or Npr2_AT2 or Gucy2C_AT1 or Gucy2D_AT1 or Gucy2F_AT1 or (    Gucy1A2B2_AT1    ) or (    Gucy1A2B3_AT1    ) or (    Gucy1A3B2_AT1    ) or (    Gucy1A3B3_AT1    )
rbc_reaction = model.reactions.get_by_id("GUACYC")
rbc_reaction.gene_reaction_rule = "(2974_AT1 and 2977_AT1) or (2974_AT1 and 2982_AT1) or (2977_AT1 and 2983_AT1) or (2982_AT1 and 2983_AT1) or 2984_AT1 or 2986_AT1 or 3000_AT1 or 4881_AT1 or 4882_AT1 or 4882_AT2"

# Reactions NDPK1, NDPK2, NDPK3: Recon3D top, iAB-RBC-283 bottom
# (4830_AT1 and 4831_AT1) or (4830_AT2 and 4831_AT1) or 4832_AT1 or 10201_AT1 or 29922_AT1 or 29922_AT2
# (      NME12_AT1      ) or (      NME12_AT2      ) or NME3_AT1 or  NME6_AT1 or  NME7_AT1 or  NME7_AT2

rbc_reaction = model.reactions.get_by_id("NDPK1")
rbc_reaction.gene_reaction_rule = "(4830_AT1 and 4831_AT1) or (4830_AT2 and 4831_AT1) or 4832_AT1 or 10201_AT1 or 29922_AT1 or 29922_AT2"
# old_genes.difference_update(rbc_reaction.genes)
rbc_reaction = model.reactions.get_by_id("NDPK2")
rbc_reaction.gene_reaction_rule = "(4830_AT1 and 4831_AT1) or (4830_AT2 and 4831_AT1) or 4832_AT1 or 10201_AT1 or 29922_AT1 or 29922_AT2"

rbc_reaction = model.reactions.get_by_id("NDPK3")
rbc_reaction.gene_reaction_rule = "(4830_AT1 and 4831_AT1) or (4830_AT2 and 4831_AT1) or 4832_AT1 or 10201_AT1 or 29922_AT1 or 29922_AT2"

# Reaction METAT: Recon3D top, iAB-RBC-283 bottom
#  4143_AT1 or (4144_AT1 and 27430_AT1) or (4144_AT1 and 27430_AT2)
# Mat1a_AT1 or (       Mat2ab1        ) or (       Mat2ab2        )
rbc_reaction = model.reactions.get_by_id("METAT")
rbc_reaction.gene_reaction_rule = "4143_AT1 or (4144_AT1 and 27430_AT1) or (4144_AT1 and 27430_AT2)"

# Reaction NaKt: Recon3D (modified due to differences), iAB-RBC-283 bottom
# (476_AT1 and 481_AT1) or (476_AT1 and  482_AT1) or (476_AT1 and 483_AT1) or (476_AT1 and 23439_AT1)
# (     Atp1a1b1      ) or (     Atp1a1b2       ) or (     Atp1a1b3      ) or (     Atp1a1b4      )
rbc_reaction = model.reactions.get_by_id("NaKt")
old_genes = set(rbc_reaction.genes)
rbc_reaction.gene_reaction_rule = "(476_AT1 and 481_AT1) or (476_AT1 and 482_AT1) or (476_AT1 and 483_AT1) or (476_AT1 and 23439_AT1)"

# Remove old lumped genes from model
cobra.manipulation.delete.remove_genes(
    model,
    gene_list=model.genes.query(lambda x: not x.reactions),
    remove_reactions=False
)
model.genes.query(lambda x: not x.reactions)

[]

#### Fix gene reaction rules with duplicate genes

In [14]:
# Auto fixed
for reaction in model.reactions.query(lambda x: x.gene_reaction_rule):
    or_split = reaction.gene_reaction_rule.split(" or ")
    if Counter(or_split) != Counter(set(or_split)):
        # Identify extra gene
        extra = Counter(or_split) - Counter(set(or_split))
        if extra:
            print(f"Reaction: {reaction.id}: {extra}")
        for gene, num_extra in extra.items():
            reaction.gene_reaction_rule = reaction.gene_reaction_rule.replace(f"{gene} or ", "", num_extra)
            
# # DAGK reactions have an extra gene
# # Manual fix
# model.reactions.get_by_id("DAGK_hs_16_0_16_0").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_16_0_18_1").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_16_0_18_2").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_18_1_18_1").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_18_1_18_2").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_18_2_16_0").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"
# model.reactions.get_by_id("DAGK_hs_18_2_18_1").gene_reaction_rule = "1606_AT1 or 1607_AT1 or 1607_AT2 or 1608_AT1 or 1609_AT1 or 8525_AT1 or 8525_AT2 or 8525_AT3 or 8526_AT1 or 8527_AT1 or 8527_AT2 or 9162_AT1 or 160851_AT1 or 160851_AT2"

Reaction: DAGK_hs_16_0_16_0: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_16_0_18_1: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_16_0_18_2: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_18_1_18_1: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_18_1_18_2: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_18_2_16_0: Counter({'1607_AT1': 1})
Reaction: DAGK_hs_18_2_18_1: Counter({'1607_AT1': 1})


#### Update annotations
##### Same as Recon3D

In [15]:
refseq_updates = {
    "2982_AT1": "GUCY1A1", # GUCY1A3 (past), (GUCY1A1 now) 
    "2983_AT1": "GUCY1B1", # GUCY1B3 (past), (GUCY1B1 now)
}
for gene_id in [
    "79178_AT1",  # now THTPA   (was ThmtP_AT1)
    "51005_AT1",  # now AMDHD2  (was Cgi_14_AT1)
    "2977_AT1",   # now GUCY1A2 (was lumped)
    "4830_AT1",   # now NME1    (was lumped)
    "4830_AT2",   # now NME1    (was lumped)
    "4831_AT1",   # now NME2    (was lumped)
    "4144_AT1",   # now MAT2A   (was lumped)  # Check refseq names from here below
    "27430_AT1",  # now MAT2B   (was lumped)
    "27430_AT2",  # now MAT2B   (was lumped)
    "476_AT1",    # now ATP1A1  (was lumped)
    "481_AT1",    # now ATP1B1  (was lumped)
    "482_AT1",    # now ATP1B2  (was lumped)
    "483_AT1",    # now ATP1B3  (was lumped)
    "23439_AT1",  # now ATP1B4  (was lumped)
    "6005_AT1",   # now RHAG
    "57127_AT1",  # now RHBG
    "10257_AT1",  # now ABCC4
    "6513_AT1",   # now SLC2A1
    "6514_AT1",   # now SLC2A2
    "6515_AT1",   # now SLC2A3
    "6517_AT1",   # now SLC2A4
    "6518_AT1",   # now SLC2A5
    "155184_AT1", # now SLC2A7
    "29988_AT1",  # now SLC2A8
    "66035_AT1",  # now SLC2A11
    "6521_AT1",   # now SLC4A1
    "6523_AT1",   # now SLC5A1
    "6526_AT1",   # now SLC5A3
    "6528_AT1",   # now SLC5A5
    "10723_AT1",  # now SLC12A7
    "6563_AT1",   # now SLC14A1
    "2030_AT1",   # now SLC29A1
    "3177_AT1",   # now SLC29A2
    "80201_AT1",  # now HKDC1

] + list(refseq_updates):
    recon_gene = Recon3D.genes.get_by_id(gene_id)
    rbc_gene = model.genes.get_by_id(gene_id)
    rbc_gene.annotation.update(recon_gene.annotation)
    if gene_id in refseq_updates:
        rbc_gene.annotation["refseq_name"] = refseq_updates[gene_id]

##### Manually added

In [16]:
rbc_gene = model.genes.get_by_id("2713_AT1") # GK3P, Pseudogene
rbc_gene.annotation.update({
    'sbo': 'SBO:0000243',
    'ccds': [],
    'ncbigene': '2713',
    'omim': '600149',
    'refseq_name': 'GK3P',
    'refseq_synonym': ['GKP3', 'GKTB']
})

rbc_gene = model.genes.get_by_id("2974_AT1") # GUCY1B2, Pseudogene
rbc_gene.annotation.update({
    'sbo': 'SBO:0000243',
    'ccds': [],
    'ncbigene': '2974',
    'omim': '603695',
    'refseq_name': 'GUCY1B2',
    'refseq_synonym': ['GC-SB2']
}) # Nothing to update from RECON3D, needs manual populating

#### Update names

In [17]:
for gene in model.genes:
    name = gene.annotation.get("refseq_name")
    if isinstance(name, list):
        if len(name) > 1:
            raise ValueError
        name = name[0]
    if name is None:
        print(gene)
        continue
    gene.name = name

## Update/Fix IDs
1. Update the identifiers of lipids in the model to match those used in the publication of iAB_RBC_283 for later comparison. 

In [18]:
model.reactions.get_by_id("AGPAT1_16_0_16_1").id = "AGPAT1_16_0_16_0"
model.reactions.get_by_id("AGPAT1_16_0_18_3").id = "AGPAT1_16_0_18_1"
model.reactions.get_by_id("AGPAT1_16_0_18_4").id = "AGPAT1_16_0_18_2"
model.reactions.get_by_id("AGPAT1_18_1_18_3").id = "AGPAT1_18_1_18_1"
model.reactions.get_by_id("AGPAT1_18_1_18_4").id = "AGPAT1_18_1_18_2"
model.reactions.get_by_id("AGPAT1_18_2_16_0").id = "AGPAT1_18_2_16_0"
model.reactions.get_by_id("AGPAT1_18_2_18_1").id = "AGPAT1_18_2_18_1"

model.reactions.get_by_id("GPAM_hs_16_1").id = "GPAM_hs_16_0"
model.reactions.get_by_id("GPAM_hs_18_3").id = "GPAM_hs_18_1"
model.reactions.get_by_id("GPAM_hs_18_4").id = "GPAM_hs_18_2"

model.reactions.get_by_id("PI4P5K_16_0_16_1").id = "PI4P5K_16_0_16_0"
model.reactions.get_by_id("PI4P5K_16_0_18_3").id = "PI4P5K_16_0_18_1"
model.reactions.get_by_id("PI4P5K_16_0_18_4").id = "PI4P5K_16_0_18_2"
model.reactions.get_by_id("PI4P5K_18_1_18_3").id = "PI4P5K_18_1_18_1"
model.reactions.get_by_id("PI4P5K_18_1_18_4").id = "PI4P5K_18_1_18_2"

model.repair()

#### Sort GPRs in reactions (optional)

In [19]:
from collections import defaultdict
or_gpr_reactions = [
    r for r in model.reactions 
    if "or" in r.gene_reaction_rule and not "and" in r.gene_reaction_rule]

and_gpr_reactions = [
    r for r in model.reactions 
    if "and" in r.gene_reaction_rule and not "or" in r.gene_reaction_rule]

for join_str, reactions in zip([" or ", " and "], [or_gpr_reactions, and_gpr_reactions]):
    for r in reactions:
        gpr_dict = defaultdict(list)
        for i in r.gene_reaction_rule.split(join_str):
            gpr_dict[int(i.split("_")[0])] += [i.split("_")[1]]
        genes = r.genes
        r.gene_reaction_rule = join_str.join([f"{key}_{v}" for key in sorted(gpr_dict) for v in sorted(gpr_dict[key])])
        try:
            assert genes == r.genes
        except AssertionError:
            raise AssertionError(f"A change occured in the genes for reaction {r.id} that should not have occured.")
    


### Update missing formula and charges
1. Extracellular metabolites do not have formulas or charges, rendering them mass-imbalanced. Use intracellular metabolite formula and charges when possible, balance out the rest assuming iAB-RBC-283 is already mass and charge balanced
2. The reaction `DM_nadh` will not be charge balanced as its a pseudoreaction.

### Using intracellular metabolites

In [20]:
# Missing formulas
model.metabolites.get_by_id('3moxtyr_e').formula = model.metabolites.get_by_id('3moxtyr_c').formula
model.metabolites.get_by_id('5aop_e').formula = model.metabolites.get_by_id('5aop_c').formula
model.metabolites.get_by_id('acnam_e').formula = model.metabolites.get_by_id('acnam_c').formula
model.metabolites.get_by_id('etha_e').formula = model.metabolites.get_by_id('etha_c').formula
model.metabolites.get_by_id('fum_e').formula = model.metabolites.get_by_id('fum_c').formula
model.metabolites.get_by_id('hcys__L_e').formula = model.metabolites.get_by_id('hcys__L_c').formula
model.metabolites.get_by_id('leuktrA4_e').formula = model.metabolites.get_by_id('leuktrA4_c').formula
model.metabolites.get_by_id('leuktrB4_e').formula = model.metabolites.get_by_id('leuktrB4_c').formula
model.metabolites.get_by_id('mal__L_e').formula = model.metabolites.get_by_id('mal__L_c').formula
model.metabolites.get_by_id('normete__L_e').formula = model.metabolites.get_by_id('normete__L_c').formula
model.metabolites.get_by_id('orot_e').formula = model.metabolites.get_by_id('orot_c').formula
model.metabolites.get_by_id('ptrc_e').formula = model.metabolites.get_by_id('ptrc_c').formula
model.metabolites.get_by_id('spmd_e').formula = model.metabolites.get_by_id('spmd_c').formula
model.metabolites.get_by_id('sprm_e').formula = model.metabolites.get_by_id('sprm_c').formula

# Missing charges
model.metabolites.get_by_id('3moxtyr_e').charge = model.metabolites.get_by_id('3moxtyr_c').charge
model.metabolites.get_by_id('5aop_e').charge = model.metabolites.get_by_id('5aop_c').charge
model.metabolites.get_by_id('acnam_e').charge = model.metabolites.get_by_id('acnam_c').charge
model.metabolites.get_by_id('etha_e').charge = model.metabolites.get_by_id('etha_c').charge
model.metabolites.get_by_id('fum_e').charge = model.metabolites.get_by_id('fum_c').charge
model.metabolites.get_by_id('hcys__L_e').charge = model.metabolites.get_by_id('hcys__L_c').charge
model.metabolites.get_by_id('leuktrA4_e').charge = model.metabolites.get_by_id('leuktrA4_c').charge
model.metabolites.get_by_id('leuktrB4_e').charge = model.metabolites.get_by_id('leuktrB4_c').charge
model.metabolites.get_by_id('mal__L_e').charge = model.metabolites.get_by_id('mal__L_c').charge
model.metabolites.get_by_id('normete__L_e').charge = model.metabolites.get_by_id('normete__L_c').charge
model.metabolites.get_by_id('orot_e').charge = model.metabolites.get_by_id('orot_c').charge
model.metabolites.get_by_id('ptrc_e').charge = model.metabolites.get_by_id('ptrc_c').charge
model.metabolites.get_by_id('spmd_e').charge = model.metabolites.get_by_id('spmd_c').charge
model.metabolites.get_by_id('sprm_e').charge = model.metabolites.get_by_id('sprm_c').charge

In [21]:
# Missing formulas
model.metabolites.get_by_id('alpa_hs_16_0_c').formula = "C19H37O7P"
model.metabolites.get_by_id('alpa_hs_18_1_c').formula = "C21H39O7P"
model.metabolites.get_by_id('alpa_hs_18_2_c').formula = "C21H37O7P"
model.metabolites.get_by_id('band_c').formula = "XH"
model.metabolites.get_by_id('bandmt_c').formula = "XCH3"
model.metabolites.get_by_id('cdpdag_hs_16_0_16_0_c').formula = "C44H79N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_16_0_18_1_c').formula = "C46H81N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_16_0_18_2_c').formula = "C46H79N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_18_1_18_1_c').formula = "C48H83N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_18_1_18_2_c').formula = "C48H81N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_18_2_16_0_c').formula = "C46H79N3O15P2"
model.metabolites.get_by_id('cdpdag_hs_18_2_18_1_c').formula = "C48H81N3O15P2"
model.metabolites.get_by_id('dag_hs_16_0_16_0_c').formula = "C35H68O5"
model.metabolites.get_by_id('dag_hs_16_0_18_1_c').formula = "C37H70O5"
model.metabolites.get_by_id('dag_hs_16_0_18_2_c').formula = "C37H68O5"
model.metabolites.get_by_id('dag_hs_18_1_18_1_c').formula = "C39H72O5"
model.metabolites.get_by_id('dag_hs_18_1_18_2_c').formula = "C39H70O5"
model.metabolites.get_by_id('dag_hs_18_2_16_0_c').formula = "C37H68O5"
model.metabolites.get_by_id('dag_hs_18_2_18_1_c').formula = "C39H70O5"
model.metabolites.get_by_id('dhmtp_c').formula = "C6H10O3S"
model.metabolites.get_by_id('lpchol_hs_16_0_c').formula = "C24H50NO7P"
model.metabolites.get_by_id('lpchol_hs_18_1_c').formula = "C26H52NO7P"
model.metabolites.get_by_id('lpchol_hs_18_2_c').formula = "C26H50NO7P"
model.metabolites.get_by_id('pa_hs_16_0_16_0_c').formula = "C35H67O8P"
model.metabolites.get_by_id('pa_hs_16_0_18_1_c').formula = "C37H69O8P"
model.metabolites.get_by_id('pa_hs_16_0_18_2_c').formula = "C37H67O8P"
model.metabolites.get_by_id('pa_hs_18_1_18_1_c').formula = "C39H71O8P"
model.metabolites.get_by_id('pa_hs_18_1_18_2_c').formula = "C39H69O8P"
model.metabolites.get_by_id('pa_hs_18_2_16_0_c').formula = "C37H67O8P"
model.metabolites.get_by_id('pa_hs_18_2_18_1_c').formula = "C39H69O8P"
model.metabolites.get_by_id('pail45p_hs_16_0_16_0_c').formula = "C41H76O19P3"
model.metabolites.get_by_id('pail45p_hs_16_0_18_1_c').formula = "C43H78O19P3"
model.metabolites.get_by_id('pail45p_hs_16_0_18_2_c').formula = "C43H76O19P3"
model.metabolites.get_by_id('pail45p_hs_18_1_18_1_c').formula = "C45H80O19P3"
model.metabolites.get_by_id('pail45p_hs_18_1_18_2_c').formula = "C45H78O19P3"
model.metabolites.get_by_id('pail45p_hs_18_2_16_0_c').formula = "C43H76O19P3"
model.metabolites.get_by_id('pail45p_hs_18_2_18_1_c').formula = "C45H78O19P3"
model.metabolites.get_by_id('pail4p_hs_16_0_16_0_c').formula = "C41H77O16P2"
model.metabolites.get_by_id('pail4p_hs_16_0_18_1_c').formula = "C43H79O16P2"
model.metabolites.get_by_id('pail4p_hs_16_0_18_2_c').formula = "C43H77O16P2"
model.metabolites.get_by_id('pail4p_hs_18_1_18_1_c').formula = "C45H81O16P2"
model.metabolites.get_by_id('pail4p_hs_18_1_18_2_c').formula = "C45H79O16P2"
model.metabolites.get_by_id('pail4p_hs_18_2_16_0_c').formula = "C43H77O16P2"
model.metabolites.get_by_id('pail4p_hs_18_2_18_1_c').formula = "C45H79O16P2"
model.metabolites.get_by_id('pail_hs_16_0_16_0_c').formula = "C41H78O13P"
model.metabolites.get_by_id('pail_hs_16_0_18_1_c').formula = "C43H80O13P"
model.metabolites.get_by_id('pail_hs_16_0_18_2_c').formula = "C43H78O13P"
model.metabolites.get_by_id('pail_hs_18_1_18_1_c').formula = "C45H82O13P"
model.metabolites.get_by_id('pail_hs_18_1_18_2_c').formula = "C45H80O13P"
model.metabolites.get_by_id('pail_hs_18_2_16_0_c').formula = "C43H78O13P"
model.metabolites.get_by_id('pail_hs_18_2_18_1_c').formula = "C45H80O13P"
model.metabolites.get_by_id('pchol_hs_16_0_16_0_c').formula = "C40H80NO8P"
model.metabolites.get_by_id('pchol_hs_16_0_18_1_c').formula = "C42H82NO8P"
model.metabolites.get_by_id('pchol_hs_16_0_18_2_c').formula = "C42H80NO8P"
model.metabolites.get_by_id('pchol_hs_18_1_18_1_c').formula = "C44H84NO8P"
model.metabolites.get_by_id('pchol_hs_18_1_18_2_c').formula = "C44H82NO8P"
model.metabolites.get_by_id('pchol_hs_18_2_16_0_c').formula = "C42H80NO8P"
model.metabolites.get_by_id('pchol_hs_18_2_18_1_c').formula = "C44H82NO8P"
model.metabolites.get_by_id('pe_hs_16_0_16_0_c').formula = "C37H74NO8P"
model.metabolites.get_by_id('pe_hs_16_0_18_1_c').formula = "C39H76NO8P"
model.metabolites.get_by_id('pe_hs_16_0_18_2_c').formula = "C39H74NO8P"
model.metabolites.get_by_id('pe_hs_18_1_18_1_c').formula = "C41H78NO8P"
model.metabolites.get_by_id('pe_hs_18_1_18_2_c').formula = "C41H76NO8P"
model.metabolites.get_by_id('pe_hs_18_2_16_0_c').formula = "C39H74NO8P"
model.metabolites.get_by_id('pe_hs_18_2_18_1_c').formula = "C41H76NO8P"
model.metabolites.get_by_id('ppp9_c').formula = "C34H32N4O4"

# Missing charges
model.metabolites.get_by_id('alpa_hs_16_0_c').charge = -2
model.metabolites.get_by_id('alpa_hs_18_1_c').charge = -2
model.metabolites.get_by_id('alpa_hs_18_2_c').charge = -2
model.metabolites.get_by_id('band_c').charge = 0
model.metabolites.get_by_id('bandmt_c').charge = 0
model.metabolites.get_by_id('cdpdag_hs_16_0_16_0_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_16_0_18_1_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_16_0_18_2_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_18_1_18_1_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_18_1_18_2_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_18_2_16_0_c').charge = -2
model.metabolites.get_by_id('cdpdag_hs_18_2_18_1_c').charge = -2
model.metabolites.get_by_id('dag_hs_16_0_16_0_c').charge = 0
model.metabolites.get_by_id('dag_hs_16_0_18_1_c').charge = 0
model.metabolites.get_by_id('dag_hs_16_0_18_2_c').charge = 0
model.metabolites.get_by_id('dag_hs_18_1_18_1_c').charge = 0
model.metabolites.get_by_id('dag_hs_18_1_18_2_c').charge = 0
model.metabolites.get_by_id('dag_hs_18_2_16_0_c').charge = 0
model.metabolites.get_by_id('dag_hs_18_2_18_1_c').charge = 0
model.metabolites.get_by_id('dhmtp_c').charge = 0
model.metabolites.get_by_id('lpchol_hs_16_0_c').charge = 0
model.metabolites.get_by_id('lpchol_hs_18_1_c').charge = 0
model.metabolites.get_by_id('lpchol_hs_18_2_c').charge = 0
model.metabolites.get_by_id('pa_hs_16_0_16_0_c').charge = -2
model.metabolites.get_by_id('pa_hs_16_0_18_1_c').charge = -2
model.metabolites.get_by_id('pa_hs_16_0_18_2_c').charge = -2
model.metabolites.get_by_id('pa_hs_18_1_18_1_c').charge = -2
model.metabolites.get_by_id('pa_hs_18_1_18_2_c').charge = -2
model.metabolites.get_by_id('pa_hs_18_2_16_0_c').charge = -2
model.metabolites.get_by_id('pa_hs_18_2_18_1_c').charge = -2
model.metabolites.get_by_id('pail45p_hs_16_0_16_0_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_16_0_18_1_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_16_0_18_2_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_18_1_18_1_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_18_1_18_2_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_18_2_16_0_c').charge = -5
model.metabolites.get_by_id('pail45p_hs_18_2_18_1_c').charge = -5
model.metabolites.get_by_id('pail4p_hs_16_0_16_0_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_16_0_18_1_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_16_0_18_2_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_18_1_18_1_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_18_1_18_2_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_18_2_16_0_c').charge = -3
model.metabolites.get_by_id('pail4p_hs_18_2_18_1_c').charge = -3
model.metabolites.get_by_id('pail_hs_16_0_16_0_c').charge = -1
model.metabolites.get_by_id('pail_hs_16_0_18_1_c').charge = -1
model.metabolites.get_by_id('pail_hs_16_0_18_2_c').charge = -1
model.metabolites.get_by_id('pail_hs_18_1_18_1_c').charge = -1
model.metabolites.get_by_id('pail_hs_18_1_18_2_c').charge = -1
model.metabolites.get_by_id('pail_hs_18_2_16_0_c').charge = -1
model.metabolites.get_by_id('pail_hs_18_2_18_1_c').charge = -1
model.metabolites.get_by_id('pchol_hs_16_0_16_0_c').charge = 0
model.metabolites.get_by_id('pchol_hs_16_0_18_1_c').charge = 0
model.metabolites.get_by_id('pchol_hs_16_0_18_2_c').charge = 0
model.metabolites.get_by_id('pchol_hs_18_1_18_1_c').charge = 0
model.metabolites.get_by_id('pchol_hs_18_1_18_2_c').charge = 0
model.metabolites.get_by_id('pchol_hs_18_2_16_0_c').charge = 0
model.metabolites.get_by_id('pchol_hs_18_2_18_1_c').charge = 0
model.metabolites.get_by_id('pe_hs_16_0_16_0_c').charge = 0
model.metabolites.get_by_id('pe_hs_16_0_18_1_c').charge = 0
model.metabolites.get_by_id('pe_hs_16_0_18_2_c').charge = 0
model.metabolites.get_by_id('pe_hs_18_1_18_1_c').charge = 0
model.metabolites.get_by_id('pe_hs_18_1_18_2_c').charge = 0
model.metabolites.get_by_id('pe_hs_18_2_16_0_c').charge = 0
model.metabolites.get_by_id('pe_hs_18_2_18_1_c').charge = 0
model.metabolites.get_by_id('ppp9_c').charge = -2

## Export updated model

In [31]:
model.id = "iAB_RBC_283"
write_sbml_model(model, f"{MAIN_PROJECT_DIR}/models/cobra/{model.id}.xml")

In [32]:

# For archival
model.id = "iAB_RBC_283_cleaned"
write_sbml_model(model, f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/{model.id}.xml")
save_json_model(model, f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/{model.id}.json")
save_yaml_model(model, f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/{model.id}.yaml")
save_matlab_model(model, f"{MAIN_PROJECT_DIR}/models/Archived/iAB_RBC_283/{model.id}.mat")